In [1]:
import CloudEnvModule
import importlib
importlib.reload(CloudEnvModule) # update module in case of changes
from CloudEnvModule import CloudEnv # import custom environment
from CloudEnvModule import STOP, calculate_evenness, CpuMem
from gym.spaces import Box
import numpy as np

In [2]:
class NewCloudEnv(CloudEnv):
    def __init__(self):
        super().__init__(n=30, m=3)
        self.timestamp = 0

    # def get_state(self):
    #     state = np.zeros(shape=(self.n, self.m, 5))
    #     for i in range(self.n):
    #         for j in range(self.m):
    #             state[i][j] = [
    #                 self.vms[i].cpu,
    #                 self.vms[i].mem,
    #                 self.servers[j].cpu,
    #                 self.servers[j].mem,
    #                 int(self.vms[i] <= self.servers[j])
    #             ]
    #     return state

    def get_reward(self):
        ans = 0
        ans -= self.timestamp
        for i in range(self.n):
            if self.vms[i].cpu == STOP:
                ans += 1
        ans *= 0.1
        ans += calculate_evenness(self.servers)
        return self.update_score(ans)

    def step(self, action):
        self.timestamp += 1
        return super().step(action)
    
    def reset(self):
        self.timestamp = 0
        return super().reset()

env = NewCloudEnv()
# env = CloudEnv()

In [3]:
# from stable_baselines3 import A2C, TD3
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike
import os
from ray.rllib.agents.ppo.ppo import PPOTrainer

In [4]:
log_path = os.path.join('Training', 'Logs')
load_path = os.path.join('Training', 'Saved Models', 'A2Cn30m3_9M')
# model = A3CTrainer.load(load_path, env=env)
from ray.tune.registry import register_env
register_env("NewCloudEnv", NewCloudEnv)
config = PPOTrainer.get_default_config()
config["env"] = "NewCloudEnv"
model = PPOTrainer(config=config)

(raylet) *** SIGSEGV received at time=1654407578 on cpu 1 ***
(raylet) PC: @     0x561f5986a8a0  (unknown)  ray::rpc::RequestWorkerLeaseReply::ByteSizeLong()
(raylet)     @     0x7f96bddf0420  281980512  (unknown)
(raylet)     @     0x561f59668438        208  grpc::GenericSerialize<>()
(raylet)     @     0x561f59668728        160  ray::rpc::ServerCallImpl<>::SendReply()
(raylet)     @     0x561f59668964        144  std::_Function_handler<>::_M_invoke()
(raylet)     @     0x561f596e55cc        496  std::_Function_handler<>::_M_invoke()
(raylet)     @     0x561f597c215a        144  std::_Function_handler<>::_M_invoke()
(raylet)     @     0x561f59708229        304  ray::raylet::LocalTaskManager::Dispatch()
(raylet)     @     0x561f5970a6df        416  ray::raylet::LocalTaskManager::PoppedWorkerHandler()
(raylet)     @     0x561f5970aba1         80  std::_Function_handler<>::_M_invoke()
(raylet)     @     0x561f596a3789        144  ray::raylet::WorkerPool::InvokePopWorkerCallbackForProcess

RayActorError: The actor died unexpectedly before finishing this task.
	class_name: RolloutWorker
	actor_id: c8cfa1a9559eea0b7c569c2201000000
	namespace: 59677e0e-60cf-46e3-b165-8b37a5f6ad38
The actor is dead because its owner has died. Owner Id: 01000000ffffffffffffffffffffffffffffffffffffffffffffffff Owner Ip address: 172.28.104.170 Owner worker exit type: NODE_DIED

: 

In [ ]:
save_path = os.path.join('Training', 'Saved Models', 'A2Cn30m3_15M')
model.save(save_path)